#Generate clips
Generate 2-sec + call length clips along with mask files (0's where no call, 1's where call) and save to use as training data. Use all labeled files in round 1.

Output clips to /content/drive/My Drive/MeerkatCallDetector/clips

Save in format:
calltype##_aud.npy
calltype##_mask.npy

In [10]:
#Generate samples for use as training data
#Generate files and store in clips folder

#params
base_dir = '/home/arianasp/meerkat_detector'
ari_labels_only = False
load_labels = True
generate_clips = True

#Directories
ground_truth_dir = base_dir + '/ground_truth'
code_dir = base_dir + '/dev'
audio_dir = base_dir + '/data/full_recordings'

#Leave one file out for later validation
validation_file = 'HM_PET_R11_20170903-20170908_file_2_(2017_09_03-05_44_59)_ASWMUX221163_SS'

#import libraries
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
import re
import IPython.display as ipd

#Set path
sys.path.append(code_dir)

#Import library
from meerkat_call_detector_library import *

if(load_labels):

  #Read in all training data
  ground_truth_files = os.listdir(ground_truth_dir)

  #Remove sound focals
  ground_truth_files = [s for s in ground_truth_files if(not('SOUNDFOC' in s))]
    
  #Remove data from validation file (will be used later)
  ground_truth_files = [s for s in ground_truth_files if(not(validation_file in s))]

  #Remove files lableed by other peoples (to use as testing data)
  if(ari_labels_only):
    ground_truth_files = [s for s in ground_truth_files if(("label.csv" in s) or ('label.CSV' in s))]

  labels_all = pandas.DataFrame()

  for file in ground_truth_files:

    #Read in curent labels for each file
    labels = pandas.read_csv(ground_truth_dir + '/' + file,delimiter='\t')

    #convert times to seconds
    labels['start_time'] = labels['Start'].map(lambda x: hms_to_seconds(x))
    labels['duration'] = labels['Duration'].map(lambda x: hms_to_seconds(x))
    labels['end_time'] = labels['start_time'] + labels['duration']

    #Add columns for label types
    labels['label_type'] = labels['Name'].apply(label_type)
    labels['call_type'] = labels['Name'].apply(call_type_simple)
    labels['focal'] = labels['Name'].apply(caller)

    #Add column for filename
    labels['file'] = file

    #Add None where appropriate
    labels.loc[labels['label_type'] != 'call','call_type'] = None
    labels.loc[labels['label_type'] != 'call','focal'] = None

    labels_all = labels_all.append(labels,ignore_index=True)

  labels = labels_all
  
if(generate_clips):

  #generate 2 sec clips and save to clips folder
  clips_dir = base_dir + '/clips'
  samprate = 8000 

  n_labels = len(labels.index)

  for i in labels.index:
    print(str(i) + '/' + str(n_labels))
    call_type = labels['call_type'][i]
    if(call_type != None):
        t0 = labels['start_time'][i] - 1
        tf = labels['end_time'][i] + 1
        file = labels['file'][i]
        aud, sr = librosa.core.load(path=audio_dir + '/' + file[0:(len(file)-4)] + '.wav',sr=samprate,offset=t0,duration=tf-t0)

        #get mask
        labs_curr = labels[(labels['file']==file) & (labels['start_time']>= t0) & (labels['end_time']<= tf) & (labels['call_type']!= None)]
        nrow = labs_curr.shape[0]
        mask = np.zeros(shape=len(aud))
        for j in labs_curr.index:
          t0_call = np.int(np.round((labs_curr['start_time'][j] - t0)*samprate))
          tf_call = np.int(np.round((labs_curr['end_time'][j] - t0)*samprate))
          if(t0_call < 0):
            t0_call = 0
          mask[t0_call:tf_call] = 1
        clip_file_name = call_type + str(i)
        np.save(clips_dir + '/' + clip_file_name + '_mask.npy',mask)
        np.save(clips_dir + '/' + clip_file_name + '_aud.npy',aud)

  #save list of labels
  fid = open(clips_dir + '/' + 'labels_all.pckl','wb')
  pickle.dump(labels,fid)
  fid.close()

0/16068
1/16068
2/16068
3/16068
4/16068
5/16068
6/16068
7/16068
8/16068
9/16068
10/16068
11/16068
12/16068
13/16068
14/16068
15/16068
16/16068
17/16068
18/16068
19/16068
20/16068
21/16068
22/16068
23/16068
24/16068
25/16068
26/16068
27/16068
28/16068
29/16068
30/16068
31/16068
32/16068
33/16068
34/16068
35/16068
36/16068
37/16068
38/16068
39/16068
40/16068
41/16068
42/16068
43/16068
44/16068
45/16068
46/16068
47/16068
48/16068
49/16068
50/16068
51/16068
52/16068
53/16068
54/16068
55/16068
56/16068
57/16068
58/16068
59/16068
60/16068
61/16068
62/16068
63/16068
64/16068
65/16068
66/16068
67/16068
68/16068
69/16068
70/16068
71/16068
72/16068
73/16068
74/16068
75/16068
76/16068
77/16068
78/16068
79/16068
80/16068
81/16068
82/16068
83/16068
84/16068
85/16068
86/16068
87/16068
88/16068
89/16068
90/16068
91/16068
92/16068
93/16068
94/16068
95/16068
96/16068
97/16068
98/16068
99/16068
100/16068
101/16068
102/16068
103/16068
104/16068
105/16068
106/16068
107/16068
108/16068
109/16068
110/16068


831/16068
832/16068
833/16068
834/16068
835/16068
836/16068
837/16068
838/16068
839/16068
840/16068
841/16068
842/16068
843/16068
844/16068
845/16068
846/16068
847/16068
848/16068
849/16068
850/16068
851/16068
852/16068
853/16068
854/16068
855/16068
856/16068
857/16068
858/16068
859/16068
860/16068
861/16068
862/16068
863/16068
864/16068
865/16068
866/16068
867/16068
868/16068
869/16068
870/16068
871/16068
872/16068
873/16068
874/16068
875/16068
876/16068
877/16068
878/16068
879/16068
880/16068
881/16068
882/16068
883/16068
884/16068
885/16068
886/16068
887/16068
888/16068
889/16068
890/16068
891/16068
892/16068
893/16068
894/16068
895/16068
896/16068
897/16068
898/16068
899/16068
900/16068
901/16068
902/16068
903/16068
904/16068
905/16068
906/16068
907/16068
908/16068
909/16068
910/16068
911/16068
912/16068
913/16068
914/16068
915/16068
916/16068
917/16068
918/16068
919/16068
920/16068
921/16068
922/16068
923/16068
924/16068
925/16068
926/16068
927/16068
928/16068
929/16068
930/16068


1592/16068
1593/16068
1594/16068
1595/16068
1596/16068
1597/16068
1598/16068
1599/16068
1600/16068
1601/16068
1602/16068
1603/16068
1604/16068
1605/16068
1606/16068
1607/16068
1608/16068
1609/16068
1610/16068
1611/16068
1612/16068
1613/16068
1614/16068
1615/16068
1616/16068
1617/16068
1618/16068
1619/16068
1620/16068
1621/16068
1622/16068
1623/16068
1624/16068
1625/16068
1626/16068
1627/16068
1628/16068
1629/16068
1630/16068
1631/16068
1632/16068
1633/16068
1634/16068
1635/16068
1636/16068
1637/16068
1638/16068
1639/16068
1640/16068
1641/16068
1642/16068
1643/16068
1644/16068
1645/16068
1646/16068
1647/16068
1648/16068
1649/16068
1650/16068
1651/16068
1652/16068
1653/16068
1654/16068
1655/16068
1656/16068
1657/16068
1658/16068
1659/16068
1660/16068
1661/16068
1662/16068
1663/16068
1664/16068
1665/16068
1666/16068
1667/16068
1668/16068
1669/16068
1670/16068
1671/16068
1672/16068
1673/16068
1674/16068
1675/16068
1676/16068
1677/16068
1678/16068
1679/16068
1680/16068
1681/16068
1682/16068

2337/16068
2338/16068
2339/16068
2340/16068
2341/16068
2342/16068
2343/16068
2344/16068
2345/16068
2346/16068
2347/16068
2348/16068
2349/16068
2350/16068
2351/16068
2352/16068
2353/16068
2354/16068
2355/16068
2356/16068
2357/16068
2358/16068
2359/16068
2360/16068
2361/16068
2362/16068
2363/16068
2364/16068
2365/16068
2366/16068
2367/16068
2368/16068
2369/16068
2370/16068
2371/16068
2372/16068
2373/16068
2374/16068
2375/16068
2376/16068
2377/16068
2378/16068
2379/16068
2380/16068
2381/16068
2382/16068
2383/16068
2384/16068
2385/16068
2386/16068
2387/16068
2388/16068
2389/16068
2390/16068
2391/16068
2392/16068
2393/16068
2394/16068
2395/16068
2396/16068
2397/16068
2398/16068
2399/16068
2400/16068
2401/16068
2402/16068
2403/16068
2404/16068
2405/16068
2406/16068
2407/16068
2408/16068
2409/16068
2410/16068
2411/16068
2412/16068
2413/16068
2414/16068
2415/16068
2416/16068
2417/16068
2418/16068
2419/16068
2420/16068
2421/16068
2422/16068
2423/16068
2424/16068
2425/16068
2426/16068
2427/16068

3082/16068
3083/16068
3084/16068
3085/16068
3086/16068
3087/16068
3088/16068
3089/16068
3090/16068
3091/16068
3092/16068
3093/16068
3094/16068
3095/16068
3096/16068
3097/16068
3098/16068
3099/16068
3100/16068
3101/16068
3102/16068
3103/16068
3104/16068
3105/16068
3106/16068
3107/16068
3108/16068
3109/16068
3110/16068
3111/16068
3112/16068
3113/16068
3114/16068
3115/16068
3116/16068
3117/16068
3118/16068
3119/16068
3120/16068
3121/16068
3122/16068
3123/16068
3124/16068
3125/16068
3126/16068
3127/16068
3128/16068
3129/16068
3130/16068
3131/16068
3132/16068
3133/16068
3134/16068
3135/16068
3136/16068
3137/16068
3138/16068
3139/16068
3140/16068
3141/16068
3142/16068
3143/16068
3144/16068
3145/16068
3146/16068
3147/16068
3148/16068
3149/16068
3150/16068
3151/16068
3152/16068
3153/16068
3154/16068
3155/16068
3156/16068
3157/16068
3158/16068
3159/16068
3160/16068
3161/16068
3162/16068
3163/16068
3164/16068
3165/16068
3166/16068
3167/16068
3168/16068
3169/16068
3170/16068
3171/16068
3172/16068

...

13221/16068
13222/16068
13223/16068
13224/16068
13225/16068
13226/16068
13227/16068
13228/16068
13229/16068
13230/16068
13231/16068
13232/16068
13233/16068
13234/16068
13235/16068
13236/16068
13237/16068
13238/16068
13239/16068
13240/16068
13241/16068
13242/16068
13243/16068
13244/16068
13245/16068
13246/16068
13247/16068
13248/16068
13249/16068
13250/16068
13251/16068
13252/16068
13253/16068
13254/16068
13255/16068
13256/16068
13257/16068
13258/16068
13259/16068
13260/16068
13261/16068
13262/16068
13263/16068
13264/16068
13265/16068
13266/16068
13267/16068
13268/16068
13269/16068
13270/16068
13271/16068
13272/16068
13273/16068
13274/16068
13275/16068
13276/16068
13277/16068
13278/16068
13279/16068
13280/16068
13281/16068
13282/16068
13283/16068
13284/16068
13285/16068
13286/16068
13287/16068
13288/16068
13289/16068
13290/16068
13291/16068
13292/16068
13293/16068
13294/16068
13295/16068
13296/16068
13297/16068
13298/16068
13299/16068
13300/16068
13301/16068
13302/16068
13303/16068
1330

13904/16068
13905/16068
13906/16068
13907/16068
13908/16068
13909/16068
13910/16068
13911/16068
13912/16068
13913/16068
13914/16068
13915/16068
13916/16068
13917/16068
13918/16068
13919/16068
13920/16068
13921/16068
13922/16068
13923/16068
13924/16068
13925/16068
13926/16068
13927/16068
13928/16068
13929/16068
13930/16068
13931/16068
13932/16068
13933/16068
13934/16068
13935/16068
13936/16068
13937/16068
13938/16068
13939/16068
13940/16068
13941/16068
13942/16068
13943/16068
13944/16068
13945/16068
13946/16068
13947/16068
13948/16068
13949/16068
13950/16068
13951/16068
13952/16068
13953/16068
13954/16068
13955/16068
13956/16068
13957/16068
13958/16068
13959/16068
13960/16068
13961/16068
13962/16068
13963/16068
13964/16068
13965/16068
13966/16068
13967/16068
13968/16068
13969/16068
13970/16068
13971/16068
13972/16068
13973/16068
13974/16068
13975/16068
13976/16068
13977/16068
13978/16068
13979/16068
13980/16068
13981/16068
13982/16068
13983/16068
13984/16068
13985/16068
13986/16068
1398

14587/16068
14588/16068
14589/16068
14590/16068
14591/16068
14592/16068
14593/16068
14594/16068
14595/16068
14596/16068
14597/16068
14598/16068
14599/16068
14600/16068
14601/16068
14602/16068
14603/16068
14604/16068
14605/16068
14606/16068
14607/16068
14608/16068
14609/16068
14610/16068
14611/16068
14612/16068
14613/16068
14614/16068
14615/16068
14616/16068
14617/16068
14618/16068
14619/16068
14620/16068
14621/16068
14622/16068
14623/16068
14624/16068
14625/16068
14626/16068
14627/16068
14628/16068
14629/16068
14630/16068
14631/16068
14632/16068
14633/16068
14634/16068
14635/16068
14636/16068
14637/16068
14638/16068
14639/16068
14640/16068
14641/16068
14642/16068
14643/16068
14644/16068
14645/16068
14646/16068
14647/16068
14648/16068
14649/16068
14650/16068
14651/16068
14652/16068
14653/16068
14654/16068
14655/16068
14656/16068
14657/16068
14658/16068
14659/16068
14660/16068
14661/16068
14662/16068
14663/16068
14664/16068
14665/16068
14666/16068
14667/16068
14668/16068
14669/16068
1467

15270/16068
15271/16068
15272/16068
15273/16068
15274/16068
15275/16068
15276/16068
15277/16068
15278/16068
15279/16068
15280/16068
15281/16068
15282/16068
15283/16068
15284/16068
15285/16068
15286/16068
15287/16068
15288/16068
15289/16068
15290/16068
15291/16068
15292/16068
15293/16068
15294/16068
15295/16068
15296/16068
15297/16068
15298/16068
15299/16068
15300/16068
15301/16068
15302/16068
15303/16068
15304/16068
15305/16068
15306/16068
15307/16068
15308/16068
15309/16068
15310/16068
15311/16068
15312/16068
15313/16068
15314/16068
15315/16068
15316/16068
15317/16068
15318/16068
15319/16068
15320/16068
15321/16068
15322/16068
15323/16068
15324/16068
15325/16068
15326/16068
15327/16068
15328/16068
15329/16068
15330/16068
15331/16068
15332/16068
15333/16068
15334/16068
15335/16068
15336/16068
15337/16068
15338/16068
15339/16068
15340/16068
15341/16068
15342/16068
15343/16068
15344/16068
15345/16068
15346/16068
15347/16068
15348/16068
15349/16068
15350/16068
15351/16068
15352/16068
1535

15953/16068
15954/16068
15955/16068
15956/16068
15957/16068
15958/16068
15959/16068
15960/16068
15961/16068
15962/16068
15963/16068
15964/16068
15965/16068
15966/16068
15967/16068
15968/16068
15969/16068
15970/16068
15971/16068
15972/16068
15973/16068
15974/16068
15975/16068
15976/16068
15977/16068
15978/16068
15979/16068
15980/16068
15981/16068
15982/16068
15983/16068
15984/16068
15985/16068
15986/16068
15987/16068
15988/16068
15989/16068
15990/16068
15991/16068
15992/16068
15993/16068
15994/16068
15995/16068
15996/16068
15997/16068
15998/16068
15999/16068
16000/16068
16001/16068
16002/16068
16003/16068
16004/16068
16005/16068
16006/16068
16007/16068
16008/16068
16009/16068
16010/16068
16011/16068
16012/16068
16013/16068
16014/16068
16015/16068
16016/16068
16017/16068
16018/16068
16019/16068
16020/16068
16021/16068
16022/16068
16023/16068
16024/16068
16025/16068
16026/16068
16027/16068
16028/16068
16029/16068
16030/16068
16031/16068
16032/16068
16033/16068
16034/16068
16035/16068
1603

In [9]:
print(len(ground_truth_files))

28
